# Collected Data

In [ ]:
Data was fetched by Kaggle and prepared for futher processing

## Input
Kaggle JSON file - authentification token

## Outputs
Generate Dataset : inputs/datasets/mildew.detection

## Additional Comments and Suggestions
None

## Import Packages

In [2]:
! pip install -r /workspace/Mildew.Detection/requirements.txt

In [3]:
import numpy
import os

## Working Directory is changed

In [5]:
current_dir = os.getcwd()
current_dir

'/workspace/Mildew.Detection/jupyter_notebooks'

In [6]:
os.chdir('/workspace/Mildew.Detection')
print("You set a new current directory")

You set a new current directory


In [7]:
current_dir = os.getcwd()
current_dir

'/workspace/Mildew.Detection'

## Installation of Kaggle

In [8]:
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 22.7 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73031 sha256=1f777d9e1074b78e058a9b39fdb8163e66640f6047aa27f63393a69015595ac7
  Stored in directory: /home/gitpod/.cache/pip/wheels/03/f3/c7/fc5a63bb33d22177609b06c5b4c714b5eb3f1b195ce9dc5e47
Successfully built kaggle


In [9]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

## Setting Kaggle Dataset and downloading data

In [11]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherry-leaves"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

 89%|█████████████████████████████████▊    | 49.0M/55.0M [00:00<00:00, 61.9MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:01<00:00, 57.7MB/s]


Unzipped Downloaded file

In [12]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')